In [1]:
import pandas as pd

In [25]:
data = pd.read_csv('../data/final_df.csv', sep=';')

# reformat date correctly
data['date'] = pd.to_datetime(data.Date, errors='coerce').dt.date

# substract season and episode number
data['Season_no'] = data['Season'].str.extract('(\d+)')
data['Episode_no'] = data['Episode'].str.extract('(\d+)')

# select top genre per season because there are double entries
show_genre = data.groupby(['Title', 'Genre']).count().Title_all.reset_index()
show_genre = show_genre.sort_values('Title_all').groupby('Title').first().reset_index(
                                    )[['Title', 'Genre']].rename({'Genre':'genre'}, axis=1)
data = data.merge(show_genre, on='Title')

# drop duplicate entries
data = data.drop_duplicates(subset=['Title', 'Season+Episode'], keep='first')

# assign Show_ID, Content_ID and Episode_ID (within a show)
data['Content_ID'] = data.index
season_id = pd.Series(data['Title'].unique()).rename('Title').reset_index().rename({'index':'Show_ID'}, axis=1)
data = data.merge(season_id, on='Title')
data['Episode_ID'] = data.sort_values(['Season_no', 'Episode']).groupby('Show_ID').cumcount() + 1

# select columns and rename
data = data[['Show_ID', 'Title', 'Content_ID', 'Episode_ID','Season_no', 'Episode_no', 'Season+Episode', 
             'Episode', 'genre', 'DurationMin', 'date', 'Year', 'Description', 'Image']]
data = data.rename({'season_no':'season','Episode_no':'Episode', 
             'Episode':'Episode_name', 'DurationMin':'Duration', 'date':'Date', 'genre':'Genre'}, axis=1)

# store processed data
data.to_csv('../data/BBC_episodes.csv', index=False)